# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys
import datetime

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [5]:
original_write = sys.stdout.write

In [6]:
print('1, 2, 3')

1, 2, 3


In [13]:
def my_write(string_text):
    now_dt = datetime.datetime.now()
    now_str = datetime.date.strftime(now_dt, format='%Y-%m-%d %H:%M:%S')
    if string_text not in [None, '\n', ' ', '\t', '']:
        new_text = '[' + now_str + ']: ' + string_text
        original_write(new_text)
        return len(new_text)
    else:
        original_write(string_text)
        return len(string_text)
    

sys.stdout.write = my_write

In [14]:
sys.stdout.write('1, 2, 3')

[2022-11-22 00:35:02]: 1, 2, 3

30

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

In [15]:
print('1, 2, 3')

[2022-11-22 00:35:04]: 1, 2, 3


In [17]:
sys.stdout.write = original_write

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [13]:
def timed_output(function):
    def wrapper(*args):
        now_dt = datetime.datetime.now()
        now_str = datetime.date.strftime(now_dt, format='%Y-%m-%d %H:%M:%S')
        extra_txt = '[' + now_str + ']: '
        new_args = [extra_txt + a for a in args]
        function(*new_args)
    return wrapper

In [14]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [15]:
print_greeting("Nikita")

Hello, [2022-11-20 23:44:15]: Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [16]:
def redirect_output(filepath):
    def decorator(func):
        def wrapper(*args):
            original_stdout = sys.stdout
            sys.stdout = open(filepath, "w")
            func(*args)
            sys.stdout.close()
            sys.stdout = original_stdout
        return wrapper
    return decorator
    

In [17]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [18]:
calculate()

In [19]:
%cat function_output.txt

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
1 4 9 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 
1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 
1 16 81 256 625 1296 2401 4096 6561 10000 14641 20736 28561 38416 50625 65536 83521 104976 130321 
